In [1]:
real_index = []
for i in range (1,41):
    j= str(i)
    index = 'data'+ j +'.avi'
    # print(index)
    real_index.append(index)

print(real_index)

['data1.avi', 'data2.avi', 'data3.avi', 'data4.avi', 'data5.avi', 'data6.avi', 'data7.avi', 'data8.avi', 'data9.avi', 'data10.avi', 'data11.avi', 'data12.avi', 'data13.avi', 'data14.avi', 'data15.avi', 'data16.avi', 'data17.avi', 'data18.avi', 'data19.avi', 'data20.avi', 'data21.avi', 'data22.avi', 'data23.avi', 'data24.avi', 'data25.avi', 'data26.avi', 'data27.avi', 'data28.avi', 'data29.avi', 'data30.avi', 'data31.avi', 'data32.avi', 'data33.avi', 'data34.avi', 'data35.avi', 'data36.avi', 'data37.avi', 'data38.avi', 'data39.avi', 'data40.avi']


In [2]:
# imports
import cv2
import mediapipe as mp
import torch
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import numpy

In [6]:
All_features = []
for k in real_index:
    print(k)
    # Load the MediaPipe face detection model
    mp_face_detection = mp.solutions.face_detection.FaceDetection()

    # Define the path of the input video file
    video_path = k

    # Create a VideoCapture object to read the video file
    cap = cv2.VideoCapture(video_path)

    # Initialize an empty list to store the detected faces
    detected_faces = []

    # Loop through the video frames and detect faces in each frame
    while True:
        # Read the next frame from the video
        ret, frame = cap.read()

        # If there are no more frames, break the loop
        if not ret:
            break

        # Convert the frame to RGB color space
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use MediaPipe face detection model to detect faces in the frame
        results = mp_face_detection.process(frame_rgb)

        # If faces are detected in the frame, save the bounding boxes of the faces
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                height, width, _ = frame.shape
                xmin = int(bbox.xmin * width)
                ymin = int(bbox.ymin * height)
                xmax = int((bbox.xmin + bbox.width) * width)
                ymax = int((bbox.ymin + bbox.height) * height)
                detected_faces.append((frame, (xmin, ymin, xmax, ymax)))


    # Release the VideoCapture object
    cap.release()

    # Set the output to be the detected faces
    step2_output = detected_faces
##########################################
    # ROI first method
    # Define the size of the ROI region
    roi_size = (80, 80)

    # Define the path of the input video file
    video_path = k

    # Create a VideoCapture object to read the video file
    cap = cv2.VideoCapture(video_path)

    # Initialize an empty list to store the ROI regions
    roi_regions = []

    # Loop through the detected faces and select the ROI region for each face
    for i, (frame, bbox) in enumerate(step2_output):
        xmin, ymin, xmax, ymax = bbox

        # Calculate the center point of the face
        center_x = int((xmin + xmax) / 2)
        center_y = int((ymin + ymax) / 2)

        # Calculate the coordinates of the ROI region
        roi_xmin = center_x - int(roi_size[0] / 2)
        roi_ymin = center_y - int(roi_size[1] / 2)
        roi_xmax = roi_xmin + roi_size[0]
        roi_ymax = roi_ymin + roi_size[1]

        # Extract the ROI region from the frame and resize it to the desired size
        roi_region = frame[roi_ymin:roi_ymax, roi_xmin:roi_xmax]
        roi_region = cv2.resize(roi_region, roi_size)

        # Append the ROI region to the list of ROI regions
        roi_regions.append(roi_region)

    # Release the VideoCapture object
    cap.release()

    # Set the output to be the ROI regions
    step3_output = roi_regions
###################################################
        #output from 14th layer of resnet 18

    resnet18 = torchvision.models.resnet18()

    # Remove the last layer of the ResNet18 model
    modules = list(resnet18.children())[: -4]   # Select up to the 14th layer
    resnet18 = torch.nn.Sequential(*modules)

    # Set the ResNet18 model to evaluation mode
    resnet18.eval()

    # Define the transform to apply to the ROI regions
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Initialize an empty list to store the features
    features = []

    # Loop through the ROI regions and extract features using the ResNet18 model
    for roi_region in step3_output:
        # Apply the transform to the ROI region
        roi_tensor = transform(roi_region).unsqueeze(0)

        # Pass the ROI tensor through the ResNet18 model
        with torch.no_grad():
            features_tensor = resnet18(roi_tensor).squeeze()

        # Append the features to the list of features
        features.append(features_tensor)

    list_of_tensors = features

    # Stack the list of tensors into a single 4D tensor
    stacked_tensor = torch.stack(list_of_tensors)

    # Print the shape of the stacked tensor
    print(stacked_tensor.shape)  # Output: torch.Size([3, 3, 4, 5])
    # print(stacked_tensor)
    All_features.append(stacked_tensor)

data1.avi
torch.Size([1205, 128, 10, 10])
data2.avi
torch.Size([1208, 128, 10, 10])
data3.avi
torch.Size([1207, 128, 10, 10])
data4.avi
torch.Size([1209, 128, 10, 10])
data5.avi
torch.Size([1210, 128, 10, 10])
data6.avi
torch.Size([1208, 128, 10, 10])
data7.avi
torch.Size([1209, 128, 10, 10])
data8.avi
torch.Size([1210, 128, 10, 10])
data9.avi
torch.Size([1225, 128, 10, 10])
data10.avi
torch.Size([1227, 128, 10, 10])
data11.avi
torch.Size([1226, 128, 10, 10])
data12.avi
torch.Size([1577, 128, 10, 10])
data13.avi
torch.Size([1221, 128, 10, 10])
data14.avi
torch.Size([1248, 128, 10, 10])
data15.avi
torch.Size([1298, 128, 10, 10])
data16.avi
torch.Size([1232, 128, 10, 10])
data17.avi
torch.Size([1226, 128, 10, 10])
data18.avi
torch.Size([1243, 128, 10, 10])
data19.avi
torch.Size([1209, 128, 10, 10])
data20.avi
torch.Size([1209, 128, 10, 10])
data21.avi
torch.Size([1208, 128, 10, 10])
data22.avi
torch.Size([1209, 128, 10, 10])
data23.avi
torch.Size([1208, 128, 10, 10])
data24.avi
torch.Siz

In [7]:
import torch
import torch.nn as nn
All_flattened = []
num_channels = All_features[0].shape[1] #(num_frames, num_channels, height, width)

for i in All_features:

    class ConvNet(nn.Module):
        def __init__(self):
            super(ConvNet, self).__init__()

            self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=16, kernel_size=3, stride=1, padding=1)
            self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
            self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

            self.flatten = nn.Flatten()

        def forward(self, x):
            x = self.conv1(x)
            x = nn.functional.relu(x)
            x = self.pool1(x)
            x = self.conv2(x)
            x = nn.functional.relu(x)
            x = self.pool2(x)
            x = self.flatten(x)

            flattened_features = x

            return flattened_features

    conv_net = ConvNet()
    stacked_tensor = i
    flattened_features = conv_net(stacked_tensor)
    print(flattened_features.shape)
    All_flattened.append(flattened_features)


torch.Size([1205, 128])
torch.Size([1208, 128])
torch.Size([1207, 128])
torch.Size([1209, 128])
torch.Size([1210, 128])
torch.Size([1208, 128])
torch.Size([1209, 128])
torch.Size([1210, 128])
torch.Size([1225, 128])
torch.Size([1227, 128])
torch.Size([1226, 128])
torch.Size([1577, 128])
torch.Size([1221, 128])
torch.Size([1248, 128])
torch.Size([1298, 128])
torch.Size([1232, 128])
torch.Size([1226, 128])
torch.Size([1243, 128])
torch.Size([1209, 128])
torch.Size([1209, 128])
torch.Size([1208, 128])
torch.Size([1209, 128])
torch.Size([1208, 128])
torch.Size([1209, 128])
torch.Size([1233, 128])
torch.Size([1230, 128])
torch.Size([1227, 128])
torch.Size([1254, 128])
torch.Size([2154, 128])
torch.Size([1231, 128])
torch.Size([1245, 128])
torch.Size([1217, 128])
torch.Size([1216, 128])
torch.Size([1224, 128])
torch.Size([1235, 128])
torch.Size([1233, 128])
torch.Size([1209, 128])
torch.Size([1233, 128])
torch.Size([1223, 128])
torch.Size([1250, 128])


In [8]:
All_flattened_size = []
for i in All_flattened:
    a = i[0:1200 , : ]
    All_flattened_size.append(a)
    # print(a.shape) #1200*128


print(len(All_flattened_size))
print(All_flattened_size[0].shape)

40
torch.Size([1200, 128])


In [9]:
HR = [84,71,70,76,73,63,61,72,75,84,67,73,68,86,91,60,64,81,69,74,83,61,83,95,90,88,66,69,96,58,56,61,76,62,88,92,63,64,60,64]
print(len(HR))
dataset = []
for i in range(0,40):
    dataset.append([All_flattened[i] ,HR[i]])
# print(dataset)
print(len(dataset))
print(dataset[0][0].shape)

40
40
torch.Size([1205, 128])


In [10]:
HR_tensor = torch.tensor(HR)
print(HR_tensor.shape)
HR_tensor_float = HR_tensor.float()

torch.Size([40])


In [11]:
data_3d = torch.stack(All_flattened_size) #(num_samples or batches, num_frames, flattened_feature_size)
print(data_3d.shape)
# data_3d
# normalize = transforms.Normalize(mean=[mean_value], std=[std_value])
# data = normalize(data)
# train_size = int(0.75 * len(data))
# test_size = len(data) - train_size
# train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])
# print(train_data)

torch.Size([40, 1200, 128])


In [12]:
data_3d_detach = data_3d.detach()
print(data_3d_detach.shape)

torch.Size([40, 1200, 128])


In [10]:
# LSTM MODEL
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

# set random seed for reproducibility
# torch.manual_seed(42)

# define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# define the hyperparameters
input_size = 12800
hidden_size = 256
num_layers = 1
output_size = 1
lr = 0.001
num_epochs = 30
batch_size = 1

# split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(data_3d_detach, HR_tensor_float, test_size=0.3, random_state=42)
# Split the data into train and test sets
num_samples = 40
train_size = int(0.7 * num_samples)
test_size = num_samples - train_size
data_train = data_3d_detach[:train_size]
data_test = data_3d_detach[train_size:]
HR_train = HR_tensor_float[:train_size]
HR_test = HR_tensor_float[train_size:]

# convert the data to PyTorch tensors
X_train = data_train
y_train = HR_train
X_test = data_test
y_test = HR_test

# create data loaders for batch processing
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# create the model and optimizer
model = LSTM(input_size, hidden_size, num_layers, output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# criterion = nn.L1Loss()
criterion = nn.MSELoss()

# train the model
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch+1, num_epochs, train_loss/len(train_loader)))

# evaluate the model
test_loss = 0.0
predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        predictions.extend(outputs.detach().numpy())
mae = np.mean(np.abs(np.array(predictions) - y_test.numpy()))
mape = np.mean(np.abs(np.array(predictions) - y_test.numpy()) / np.array(y_test.numpy()))*100

print("Test Loss: {:.4f}, MAE: {:.4f}, MAPE: {:.4f}".format(test_loss/len(test_loader), mae, mape))


C:\Users\mojta\anaconda3\envs\cudatest\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/30], Train Loss: 4447.9434
Epoch [2/30], Train Loss: 3469.2848
Epoch [3/30], Train Loss: 2724.4774
Epoch [4/30], Train Loss: 2121.7784
Epoch [5/30], Train Loss: 1635.6766
Epoch [6/30], Train Loss: 1250.7054
Epoch [7/30], Train Loss: 949.4941
Epoch [8/30], Train Loss: 717.0282
Epoch [9/30], Train Loss: 540.4717
Epoch [10/30], Train Loss: 408.7037
Epoch [11/30], Train Loss: 312.1651
Epoch [12/30], Train Loss: 242.7851
Epoch [13/30], Train Loss: 193.9014
Epoch [14/30], Train Loss: 160.0284
Epoch [15/30], Train Loss: 136.6557
Epoch [16/30], Train Loss: 121.2980
Epoch [17/30], Train Loss: 111.6035
Epoch [18/30], Train Loss: 105.5871
Epoch [19/30], Train Loss: 101.9228
Epoch [20/30], Train Loss: 99.7432
Epoch [21/30], Train Loss: 98.4822
Epoch [22/30], Train Loss: 97.7768
Epoch [23/30], Train Loss: 97.3992
Epoch [24/30], Train Loss: 97.2097
Epoch [25/30], Train Loss: 97.1247
Epoch [26/30], Train Loss: 97.0951
Epoch [27/30], Train Loss: 97.2496
Epoch [28/30], Train Loss: 97.1048
Epoc

In [17]:
# FC MODELS
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split


# # define the fully connected model 1 hidden layer
# class FC(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(FC, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, output_size)
#
#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out
#
# # define the hyperparameters
# input_size = 1200 * 128  #num_frames * flattened_feature_size
# hidden_size = 128
# output_size = 1
# lr = 0.001
# num_epochs = 30
# batch_size = 1

# define the fully connected model 3 hidden layers
class FC(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(FC, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        return out

# define the hyperparameters
input_size = 1200 *128
hidden_size1 = 1024
hidden_size2 = 512
hidden_size3 = 64
output_size = 1
lr = 0.001
num_epochs = 50
batch_size = 2


# reshape the data
data_2d = data_3d_detach.reshape(-1, 128 * 1200)

# split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(data_2d, HR_tensor_float, test_size=0.3, random_state=42)
num_samples = 40
train_size = int(0.7 * num_samples)
test_size = num_samples - train_size
data_train = data_3d_detach[:train_size]
data_test = data_3d_detach[train_size:]
HR_train = HR_tensor_float[:train_size]
HR_test = HR_tensor_float[train_size:]

# convert the data to PyTorch tensors
X_train = data_train
y_train = HR_train
X_test = data_test
y_test = HR_test

# create data loaders for batch processing
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# create the model and optimizer
# model = FC(input_size, hidden_size, output_size)
model = FC(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

# train the model
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        inputs = inputs.view(inputs.size(0), -1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch+1, num_epochs, train_loss/len(train_loader)))

# evaluate the model
test_loss = 0.0
predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.view(inputs.size(0), -1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        predictions.extend(outputs.detach().numpy())
mae = np.mean(np.abs(np.array(predictions) - y_test.numpy()))
mape = np.mean(np.abs(np.array(predictions) - y_test.numpy()) / np.array(y_test.numpy()))*100

print("Test Loss: {:.4f}, MAE: {:.4f}, MAPE: {:.4f}".format(test_loss/len(test_loader), mae, mape))


C:\Users\mojta\anaconda3\envs\cudatest\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Train Loss: 3491.8925
Epoch [2/50], Train Loss: 1991.0567
Epoch [3/50], Train Loss: 1695.3430
Epoch [4/50], Train Loss: 616.2626
Epoch [5/50], Train Loss: 872.1543
Epoch [6/50], Train Loss: 504.2469
Epoch [7/50], Train Loss: 372.3413
Epoch [8/50], Train Loss: 366.6935
Epoch [9/50], Train Loss: 297.4050
Epoch [10/50], Train Loss: 243.3759
Epoch [11/50], Train Loss: 253.5947
Epoch [12/50], Train Loss: 275.6736
Epoch [13/50], Train Loss: 281.9486
Epoch [14/50], Train Loss: 204.3950
Epoch [15/50], Train Loss: 111.9254
Epoch [16/50], Train Loss: 133.3712
Epoch [17/50], Train Loss: 180.0761
Epoch [18/50], Train Loss: 143.7936
Epoch [19/50], Train Loss: 121.2187
Epoch [20/50], Train Loss: 256.3872
Epoch [21/50], Train Loss: 116.3257
Epoch [22/50], Train Loss: 111.0246
Epoch [23/50], Train Loss: 102.3407
Epoch [24/50], Train Loss: 135.1441
Epoch [25/50], Train Loss: 95.9714
Epoch [26/50], Train Loss: 114.3419
Epoch [27/50], Train Loss: 141.9577
Epoch [28/50], Train Loss: 200.1542

In [ ]:
# second LSTM just for evaluation and learning
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data_3d_detach, HR_tensor_float, test_size=0.3)

# Define LSTM model
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

input_dim = 128
hidden_dim = 64
output_dim = 1
model = LSTM(input_dim, hidden_dim, output_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train model
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), y_train)
    loss.backward()
    optimizer.step()


    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Test model
test_outputs = model(X_test)
test_loss = criterion(test_outputs.squeeze(), y_test)
print('Test MAE: {:.4f}'.format(torch.mean(torch.abs(test_outputs.squeeze() - y_test))))
print('Test MAPE: {:.4f}'.format(torch.mean(torch.abs(test_outputs.squeeze() - y_test) / y_test)))